In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json
from sklearn.metrics import mean_squared_error
import numpy as np

In [14]:
data = pd.read_csv("GAN_outputs.csv",converters={"raw": lambda x: x.strip("[]").split(", "), "generated": lambda x: x.strip("[]").split(", ")})
data.head()

,gen_emotion,text,raw,generated
0,0,Don't forget a jacket,"[0.8571428656578064, 0.0, 0.25, 0.371428579092...","[0.22702576220035553, 0.20451365411281586, 0.2..."
1,0,That is exactly what happened,"[0.0, 0.3333333432674408, 0.5471698045730591, ...","[0.03509840369224548, 0.14972323179244995, 0.6..."
2,0,The surface is slick,"[0.0833333358168602, -0.0416666679084301, -0.2...","[0.06503909081220627, 0.08630925416946411, 0.0..."
3,0,We'll stop in a couple of minutes,"[0.22727273404598236, -0.02631578966975212, 0....","[0.13837862014770508, 0.10357220470905304, 0.0..."
4,0,Don't forget a jacket,"[-0.13333334028720856, -0.2857142984867096, -0...","[-0.17102427780628204, -0.2713683843612671, -0..."


In [15]:
for index, row in data.iterrows():
    data["raw"][index] = [float(i) for i in data["raw"][index]]

C:\Users\ptut0\AppData\Local\Temp\ipykernel_12212\2571763454.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["raw"][index] = [float(i) for i in data["raw"][index]]


In [16]:
for index, row in data.iterrows():
    data["generated"][index] = [float(i) for i in data["generated"][index]]

C:\Users\ptut0\AppData\Local\Temp\ipykernel_12212\2518534346.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["generated"][index] = [float(i) for i in data["generated"][index]]


In [21]:
data["gen_emotion"][data["gen_emotion"] == 0] = "A"
data["gen_emotion"][data["gen_emotion"] == 1] = "H"
data["gen_emotion"][data["gen_emotion"] == 2] = "N"

In [22]:
data.head()

,gen_emotion,text,raw,generated
0,A,Don't forget a jacket,"[0.8571428656578064, 0.0, 0.25, 0.371428579092...","[0.22702576220035553, 0.20451365411281586, 0.2..."
1,A,That is exactly what happened,"[0.0, 0.3333333432674408, 0.5471698045730591, ...","[0.03509840369224548, 0.14972323179244995, 0.6..."
2,A,The surface is slick,"[0.0833333358168602, -0.0416666679084301, -0.2...","[0.06503909081220627, 0.08630925416946411, 0.0..."
3,A,We'll stop in a couple of minutes,"[0.22727273404598236, -0.02631578966975212, 0....","[0.13837862014770508, 0.10357220470905304, 0.0..."
4,A,Don't forget a jacket,"[-0.13333334028720856, -0.2857142984867096, -0...","[-0.17102427780628204, -0.2713683843612671, -0..."


In [7]:
sentences = {
    "IEO": "It's eleven o'clock", 
    "TIE": "That is exactly what happened",
    "IOM": "I'm on my way to the meeting",
    "IWW": "I wonder what this is about",
    "TAI": "The airplane is almost full",
    "MTI": "Maybe tomorrow it will be cold",
    "IWL": "I would like a new alarm clock",
    "ITH": "I think I have a doctor's appointment",
    "DFA": "Don't forget a jacket",
    "ITS": "I think I've seen this before",
    "TSI": "The surface is slick",
    "WSI": "We'll stop in a couple of minutes"
}

emotions = {
    "angry": "A",
    "cheerful": "H",
    "neutral": "N"
}

In [23]:
IEO = data[data["text"] == "It's eleven o'clock"]
TIE = data[data["text"] == "That is exactly what happened"]
IOM = data[data["text"] == "I'm on my way to the meeting"]
IWW = data[data["text"] == "I wonder what this is about"]
TAI = data[data["text"] == "The airplane is almost full"]
MTI = data[data["text"] == "Maybe tomorrow it will be cold"]
IWL = data[data["text"] == "I would like a new alarm clock"]
ITH = data[data["text"] == "I think I have a doctor's appointment"]
DFA = data[data["text"] == "Don't forget a jacket"]
ITS = data[data["text"] == "I think I've seen this before"]
TSI = data[data["text"] == "The surface is slick"]
WSI = data[data["text"] == "We'll stop in a couple of minutes"]

In [25]:
script_dfs = [IEO, TIE, IOM, IWW, TAI, MTI, IWL, ITH, DFA, ITS, TSI, WSI]

In [58]:
A_rmse = []
A_pcc = []
H_rmse = []
H_pcc = []
N_rmse = []
N_pcc = []

for df in script_dfs:
  A = df[df["gen_emotion"] == "A"]
  A_rmse_text = []
  A_pcc_text = []
  for i in range(len(df["raw"].iloc[0])):
    raw = []
    generated = []
    for index, row in A.iterrows():
      raw.append(round(row["raw"][i], 3))
      generated.append(round(row["generated"][i], 3))
    A_rmse_text.append(mean_squared_error(raw, generated))
    A_pcc_text.append(np.corrcoef(raw, generated)[0][1])
  A_rmse.append(np.mean(A_rmse_text))
  A_pcc.append(np.mean(A_pcc_text))
  H = df[df["gen_emotion"] == "H"]
  H_rmse_text = []
  H_pcc_text = []
  for i in range(len(df["raw"].iloc[0])):
    raw = []
    generated = []
    for index, row in H.iterrows():
      raw.append(round(row["raw"][i], 3))
      generated.append(round(row["generated"][i], 3))
    H_rmse_text.append(mean_squared_error(raw, generated))
    H_pcc_text.append(np.corrcoef(raw, generated)[0][1])
  H_rmse.append(np.mean(H_rmse_text))
  H_pcc.append(np.mean(H_pcc_text))
  N = df[df["gen_emotion"] == "N"]
  N_rmse_text = []
  N_pcc_text = []
  for i in range(len(df["raw"].iloc[0])):
    raw = []
    generated = []
    for index, row in N.iterrows():
      raw.append(round(row["raw"][i], 3))
      generated.append(round(row["generated"][i],3))
    N_rmse_text.append(mean_squared_error(raw, generated))
    N_pcc_text.append(np.corrcoef(raw, generated)[0][1])
  N_rmse.append(np.mean(N_rmse_text))
  N_pcc.append(np.mean(N_pcc_text))
A_rmse_final = np.mean(A_rmse)
A_pcc_final = np.mean(A_pcc)
H_rmse_final = np.mean(H_rmse)
H_pcc_final = np.mean(H_pcc)
N_rmse_final = np.mean(N_rmse)
N_pcc_final = np.mean(N_pcc)

{0.16104411971830987: 0.098, 0.12555334507042254: 0.387, 0.11561192957746479: 0.63}
0.16104411971830987
0.048886698113207554
0.12555334507042254
0.048886698113207554
0.11561192957746479
0.048886698113207554
0.06759684905660378
0.048886698113207554
0.05651137735849057
0.048886698113207554
0.16104411971830987
0.14033822641509436
0.12555334507042254
0.14033822641509436
0.11561192957746479
0.14033822641509436
0.06759684905660378
0.14033822641509436
0.05651137735849057
0.14033822641509436
0.16104411971830987
0.09726739622641509
0.12555334507042254
0.09726739622641509
0.11561192957746479
0.09726739622641509
0.06759684905660378
0.09726739622641509
0.05651137735849057
0.09726739622641509
{0.16104411971830987: 0.098, 0.12555334507042254: 0.387, 0.11561192957746479: 0.63, 0.06759684905660378: 0.035, 0.05651137735849057: 0.007}
0.16104411971830987
0.06007570000000001
0.12555334507042254
0.06007570000000001
0.11561192957746479
0.06007570000000001
0.06759684905660378
0.06007570000000001
0.056511377

In [51]:
print(round(A_rmse_final, 3))
print(round(A_pcc_final, 3))
print(round(H_rmse_final, 3))
print(round(H_pcc_final, 3))
print(round(N_rmse_final, 3))
print(round(N_pcc_final, 3))

0.096
0.623
0.06
0.599
0.0
nan


In [90]:
A_min_min = {}
A_max = {}
H_min_min = {}
H_max = {}
N_min_min = {}
N_max = {}
for df in script_dfs:
  A_min = {}
  H_min = {}
  N_min = {}
  A_min_min[df["text"].iloc[0]] = 999
  A_max[df["text"].iloc[0]] = 0
  A = df[df["gen_emotion"] == "A"]
  for index, row in A.iterrows():
    A_rmse_text = mean_squared_error(row["raw"], row["generated"])
    if A_rmse_text < A_min_min[df["text"].iloc[0]]:
      A_min_min[df["text"].iloc[0]] = A_rmse_text
    if A_rmse_text > A_max[df["text"].iloc[0]]:
      A_max[df["text"].iloc[0]] = A_rmse_text
    if len(A_min.keys()) <= 1:
      A_min[A_rmse_text] = row["generated"]
    else:
      for key in A_min:
        if key > A_rmse_text:
          A_min.pop(key)
          A_min[A_rmse_text] = row["generated"]
          break
  for index, key in enumerate(A_min):
    A_hold = {}
    A_hold[df["text"].iloc[0]] = A_min[key]
    with open(f'./data/angry_json/A_{index}_{df["text"].iloc[0]}.json', 'w') as fp:
      json.dump(A_hold, fp)
  H_min_min[df["text"].iloc[0]] = 999
  H_max[df["text"].iloc[0]] = 0
  H = df[df["gen_emotion"] == "A"]
  for index, row in H.iterrows():
    H_rmse_text = mean_squared_error(row["raw"], row["generated"])
    if H_rmse_text < H_min_min[df["text"].iloc[0]]:
      H_min_min[df["text"].iloc[0]] = H_rmse_text
    if H_rmse_text > H_max[df["text"].iloc[0]]:
      H_max[df["text"].iloc[0]] = H_rmse_text
    if len(H_min.keys()) <= 1:
      H_min[H_rmse_text] = row["generated"]
    else:
      for key in H_min:
        if key > H_rmse_text:
          H_min.pop(key)
          H_min[H_rmse_text] = row["generated"]
          break
  for index, key in enumerate(H_min):
    H_hold = {}
    H_hold[df["text"].iloc[0]] = H_min[key]
    with open(f'./data/happy_json/H_{index}_{df["text"].iloc[0]}.json', 'w') as fp:
      json.dump(H_hold, fp)

In [91]:
print(H_min_min)
print(H_max)
print(A_min_min)
print(A_max)

{"It's eleven o'clock": 0.00011195813177709384, 'That is exactly what happened': 0.0005513426872328526, "I'm on my way to the meeting": 0.005322818269076427, 'I wonder what this is about': 0.006347961013032063, 'The airplane is almost full': 0.0052995278774761265, 'Maybe tomorrow it will be cold': 0.0020401389527052983, 'I would like a new alarm clock': 0.004785402064411074, "I think I have a doctor's appointment": 0.010324355071906889, "Don't forget a jacket": 0.0020008871363551223, "I think I've seen this before": 0.006506917950815099, 'The surface is slick': 0.0021277187036250938, "We'll stop in a couple of minutes": 0.006419025290380266}
{"It's eleven o'clock": 1.1099299295804006, 'That is exactly what happened': 0.32899771683797324, "I'm on my way to the meeting": 1.012271254726312, 'I wonder what this is about': 0.3628130142982647, 'The airplane is almost full': 0.4904607856530262, 'Maybe tomorrow it will be cold': 0.5289676713835035, 'I would like a new alarm clock': 0.466810169